# Deterministic Unit Commitment Problem

In [25]:
import sys

sys.path.append("../sddip")

In [26]:
# %run ../scripts/create_result_directories.py

In [27]:
import os

import gurobipy as gp
import numpy as np
import pandas as pd

from sddip import config, utils

## Data Processing

In [28]:
test_case_raw_dir = "WB2/raw"

test_case_raw_dir = os.path.join(config.test_cases_dir, test_case_raw_dir)

bus_file_raw = os.path.join(test_case_raw_dir, "bus_data.txt")
branch_file_raw = os.path.join(test_case_raw_dir, "branch_data.txt")
gen_file_raw = os.path.join(test_case_raw_dir, "gen_data.txt")
gen_cost_file_raw = os.path.join(test_case_raw_dir, "gen_cost_data.txt")

scenario_data_file = os.path.join(test_case_raw_dir, "scenario_data.txt")

bus_df = pd.read_csv(bus_file_raw, delimiter="\t")
branch_df = pd.read_csv(branch_file_raw, delimiter="\t")
gen_df = pd.read_csv(gen_file_raw, delimiter="\t")
gen_cost_df = pd.read_csv(gen_cost_file_raw, delimiter="\t")

scenario_df = pd.read_csv(scenario_data_file, delimiter="\t")

In [29]:
bus_df

,bus_i,type,Pd,Qd,Gs,Bs,area,Vm,Va,baseKV,zone,Vmax,Vmin
0,1,3,0,0,0,0,1,0.964,0,0,1,1.05,0.95
1,2,1,350,-350,0,0,1,1.000,-65,0,1,1.05,0.95


In [30]:
branch_df

,fbus,tbus,r,x,b,rateA,rateB,rateC,ratio,angle,status,angmin,angmax
0,1,2,0.04,0.2,0,990000,0,0,0,0,1,-360,360


In [31]:
gen_df

,bus,Pg,Qg,Qmax,Qmin,Vg,mBase,status,Pmax,Pmin,...,Pc2,Qc1min,Qc1max,Qc2min,Qc2max,ramp_agc,ramp_10,ramp_30,ramp_q,apf
0,1,400,100,400,-400,0.964,100,1,600,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
gen_cost_df

,type,startup,shutdown,n,c2,c1,c0
0,2,0,0,3,0,2,0


In [33]:
scenario_df

,t,n,p,Pd
0,1,1,1.0,350
1,2,1,0.5,300
2,2,2,0.5,200


### Power Transfer Distribution Factor

In [34]:
nodes = bus_df.bus_i.values.tolist()
edges = branch_df[["fbus", "tbus"]].values.tolist()

graph = utils.Graph(nodes, edges)

ref_bus = bus_df.loc[bus_df.type == 3].bus_i.values[0]

a_inc = graph.incidence_matrix()
b_l = (-branch_df.x / (branch_df.r**2 + branch_df.x**2)).tolist()
b_diag = np.diag(b_l)

m1 = b_diag.dot(a_inc)
m2 = a_inc.T.dot(b_diag).dot(a_inc)

m1 = np.delete(m1, ref_bus - 1, 1)
m2 = np.delete(m2, ref_bus - 1, 0)
m2 = np.delete(m2, ref_bus - 1, 1)

ptdf = m1.dot(np.linalg.inv(m2))

zeros_col = np.zeros((1, ptdf.shape[1]))

ptdf = np.insert(ptdf, ref_bus - 1, zeros_col, axis=1)

ptdf

array([[ 0., -1.]])

### Parameter Oragnization

In [35]:
########################################################################################################################
# Deterministic parameters
########################################################################################################################
gc = np.array(gen_cost_df.c1)
suc = np.array(gen_cost_df.startup)
sdc = np.array(gen_cost_df.startup)
pg_min = np.array(gen_df.Pmin)
pg_max = np.array(gen_df.Pmax)
pl_max = np.array(branch_df.rateA)

n_gens = len(gc)
n_lines, n_buses = ptdf.shape

gens_at_bus = [[] for _ in range(n_buses)]
g = 0
for b in gen_df.bus.values:
    gens_at_bus[b - 1].append(g)
    g += 1

########################################################################################################################
# Stochastic parameters
########################################################################################################################
n_nodes_per_stage = scenario_df.groupby("t")["n"].nunique().tolist()
n_stages = len(n_nodes_per_stage)

probs = np.empty(n_stages, dtype=object)
# p_d[t,n]
p_d = np.empty(n_stages, dtype=object)

for t in range(n_stages):
    stage_df = scenario_df.loc[scenario_df["t"] == t + 1]
    probs[t] = np.array(stage_df.p)
    p_d[t] = np.array(stage_df.Pd)

########################################################################################################################
# Expected values of stochastic parameters
########################################################################################################################
ex_pd = np.array([probs[t].dot(p_d[t]) for t in range(n_stages)])

## Deterministic Problem

In [36]:
det_model = gp.Model("Deterministic Problem")


########################################################################################################################
# Add variables
########################################################################################################################
x = np.empty((n_gens, n_stages), dtype=object)
y = np.empty((n_gens, n_stages), dtype=object)
s_up = np.empty((n_gens, n_stages), dtype=object)
s_down = np.empty((n_gens, n_stages), dtype=object)

for g in range(n_gens):
    for t in range(n_stages):
        x[g, t] = det_model.addVar(
            vtype=gp.GRB.BINARY, name="x_%i,%i" % (g + 1, t + 1)
        )
        y[g, t] = det_model.addVar(
            vtype=gp.GRB.CONTINUOUS, lb=0, name="y_%i,%i" % (g + 1, t + 1)
        )
        s_up[g, t] = det_model.addVar(
            vtype=gp.GRB.BINARY, name="s_up%i,%i" % (g + 1, t + 1)
        )
        s_down[g, t] = det_model.addVar(
            vtype=gp.GRB.BINARY, name="s_down%i,%i" % (g + 1, t + 1)
        )

det_model.update()

########################################################################################################################
# Add objective
########################################################################################################################
cost_coeffs = gc.tolist() + suc.tolist() + sdc.tolist()

obj = gp.LinExpr()

for t in range(n_stages):
    obj_vars = y[:, t].tolist() + s_up[:, t].tolist() + s_down[:, t].tolist()
    obj += gp.LinExpr(cost_coeffs, obj_vars)

det_model.setObjective(obj)

########################################################################################################################
# Add constraints
########################################################################################################################
# Balance constraints
det_model.addConstrs(
    (gp.quicksum(y[:, t]) == ex_pd[t] for t in range(n_stages)), "balance"
)

# Generator constraints
for g in range(n_gens):
    det_model.addConstrs(
        (y[g, t] >= pg_min[g] * x[g, t] for t in range(n_stages)), "gen_min"
    )
    det_model.addConstrs(
        (y[g, t] <= pg_max[g] * x[g, t] for t in range(n_stages)), "gen_max"
    )

# Power flow constraints
for l in range(n_lines):
    det_model.addConstrs(
        (
            gp.quicksum(
                ptdf[l, b] * gp.quicksum(y[g, t] for g in gens_at_bus[b])
                for b in range(n_buses)
            )
            <= pl_max
            for t in range(n_stages)
        ),
        "flow(1)",
    )
    det_model.addConstrs(
        (
            -gp.quicksum(
                ptdf[l, b] * gp.quicksum(y[g, t] for g in gens_at_bus[b])
                for b in range(n_buses)
            )
            <= pl_max
            for t in range(n_stages)
        ),
        "flow(2)",
    )

# Startup and shutdown constraints
for g in range(n_gens):
    det_model.addConstr((x[g, 0] <= s_up[g, 0]), "up-down(0.1)")
    det_model.addConstr((x[g, 0] <= s_up[g, 0] - s_down[g, t]), "up-down(0.2)")
    det_model.addConstrs(
        (x[g, t] - x[g, t - 1] <= s_up[g, t] for t in range(1, n_stages)),
        "up-down(1)",
    )
    det_model.addConstrs(
        (
            x[g, t] - x[g, t - 1] <= s_up[g, t] - s_down[g, t]
            for t in range(1, n_stages)
        ),
        "up-down(2)",
    )

det_model.update()

det_model.optimize()
det_model.printAttr("X")

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 14 rows, 8 columns and 20 nonzeros
Model fingerprint: 0x886e56ee
Variable types: 2 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+02, 1e+06]
Presolve removed 14 rows and 8 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 1200 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.200000000000e+03, best bound 1.200000000000e+03, gap 0.0000%

    Variable            X 
-------------------------
       x_1,1            1 
       y_1,1          350 
     s_up1,1            1 
       x_1,2            1 
       y_1,2          250 
     s_up1